In [1]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
print("GPU available:", tf.config.list_physical_devices('GPU'))

2025-06-05 01:39:12.057054: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749087552.289413      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749087552.354574      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


TensorFlow version: 2.18.0
GPU available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
!pip install -U keras-nlp
!pip install -U pandas
!pip install -U clean-text
!pip install -U tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 876.5/876.5 kB 39.6 MB/s eta 0:00:00
  Attempting uninstall: keras-hub
    Found existing installation: keras-hub 0.18.1
    Uninstalling keras-hub-0.18.1:
      Successfully uninstalled keras-hub-0.18.1
  Attempting uninstall: keras-nlp
    Found existing installation: keras-nlp 0.18.1
    Uninstalling keras-nlp-0.18.1:
      Successfully uninstalled keras-nlp-0.18.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 9.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.7 MB/s eta 0:00:00
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171031 sha256=579de6bca4ae746494acafdbed880e11a10448fcbd97bd1efe8306a2e0ddfa6e
  Stored in directory: /root/.cache/pip/wheels/bd/22/e5/b69726d5e1a19795ecd3b3e7464b16c0f1d019aa94ff1c8578
Successfully built emoji
  Attempting uninstall: emoji
    Found existing installation: emoji 2.14.1
    Uninstalling emoji-2.14

In [3]:
import tensorflow as tf
from tensorflow import keras
import keras_nlp

import pandas as pd
from cleantext import clean
import random

from tqdm import tqdm
from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()

/tmp/ipykernel_35/2797471344.py:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`
  from tqdm._tqdm_notebook import tqdm_notebook


In [4]:
BATCH_SIZE = 64
EPOCHS = 10
MAX_SEQUENCE_LENGTH = 150
TEXT_VOCAB_SIZE = 7700
SUMM_VOCAB_SIZE = 3800

EMBED_DIM = 256
INTERMEDIATE_DIM = 2048
NUM_HEADS = 8

In [5]:
train_df = pd.read_json("/kaggle/input/dataset/train.json")
train_df.head()


,id,summary,dialogue
0,13818513,Amanda baked cookies and will bring Jerry some...,Amanda: I baked cookies. Do you want some?\r\...
1,13728867,Olivia and Olivier are voting for liberals in ...,Olivia: Who are you voting for in this electio...
2,13681000,Kim may try the pomodoro technique recommended...,"Tim: Hi, what's up?\r\nKim: Bad mood tbh, I wa..."
3,13730747,Edward thinks he is in love with Bella. Rachel...,"Edward: Rachel, I think I'm in ove with Bella...."
4,13728094,"Sam is confused, because he overheard Rick com...",Sam: hey overheard rick say something\r\nSam:...


In [6]:
print("Text:\n",train_df["dialogue"][0])
print("-----------------------------")
print("Summary: ", train_df["summary"][0])


Text:
 Amanda: I baked  cookies. Do you want some?
Jerry: Sure!
Amanda: I'll bring you tomorrow :-)
-----------------------------
Summary:  Amanda baked cookies and will bring Jerry some tomorrow.


In [7]:
def clean_samples(text):
    return clean(text=text, normalize_whitespace=True, no_urls=True, no_emoji=True)
    
text_pairs = []
for i,j in zip(train_df.dialogue, train_df.summary):
    if len(i.split(" ")) < MAX_SEQUENCE_LENGTH:
        text_pairs.append((clean_samples(i),clean_samples(j)))
# print(text_pairs)

In [8]:
def train_word_piece(text_samples, vocab_size, reserved_tokens):
    word_piece_ds = tf.data.Dataset.from_tensor_slices(text_samples)
    print(word_piece_ds)
    vocab = keras_nlp.tokenizers.compute_word_piece_vocabulary(
    word_piece_ds.batch(1000).prefetch(2),
    vocabulary_size=vocab_size,
    reserved_tokens=reserved_tokens,
    )
    return vocab


In [9]:
reserved_tokens = ["[PAD]", "[UNK]", "[START]", "[END]"]

text_samples = [text_pair[0] for text_pair in text_pairs]
# print(text_samples)
text_vocab = train_word_piece(text_samples, TEXT_VOCAB_SIZE, reserved_tokens)

summ_samples = [text_pair[1] for text_pair in text_pairs]
# print(summ_samples)
summ_vocab = train_word_piece(text_samples, SUMM_VOCAB_SIZE, reserved_tokens)

I0000 00:00:1749087590.486314      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


<_TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>
<_TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>


In [10]:
print("Text Tokens: ", text_vocab[100:110])
print("Summarization Tokens: ", summ_vocab[100:110])


Text Tokens:  ['don', 'with', 'this', '##s', 'was', 'there', 'like', 'your', 'about', 'he']
Summarization Tokens:  ['just', 'don', 'with', 'this', 'was', 'there', 'like', 'your', 'about', 'he']


In [11]:
len(text_vocab), len(summ_vocab)


(6453, 3612)

In [12]:
text_tokenizer = keras_nlp.tokenizers.WordPieceTokenizer(
    vocabulary=text_vocab,
    lowercase=False)
summ_tokenizer = keras_nlp.tokenizers.WordPieceTokenizer(
vocabulary=summ_vocab, lowercase=False
)


In [13]:
text_input_ex = text_pairs[0][0]
text_tokens_ex = text_tokenizer.tokenize(text_input_ex)
print("Text sentence: ",text_input_ex)
print("Tokens: ", text_tokens_ex)
print("Recovered text after detokenizing: ",text_tokenizer.detokenize(text_tokens_ex))
print("-"*100)
summ_input_ex = text_pairs[0][1]
summ_tokens_ex = summ_tokenizer.tokenize(summ_input_ex)
print("Summarization sentence: ", summ_input_ex)
print("Tokens: ", summ_tokens_ex)
print("Recovered text after detokenizing: ",summ_tokenizer.detokenize(summ_tokens_ex))


Text sentence:  amanda: i baked cookies. do you want some?
jerry: sure!
amanda: i'll bring you tomorrow :-)
Tokens:  tf.Tensor(
[ 410   29   49 3453 1746   17   90   70  144  123   34  572   29  115
    4  410   29   49   10   95  303   70  172   29   16   12], shape=(26,), dtype=int32)
Recovered text after detokenizing:  amanda : i baked cookies . do you want some ? jerry : sure ! amanda : i ' ll bring you tomorrow : - )
----------------------------------------------------------------------------------------------------
Summarization sentence:  amanda baked cookies and will bring jerry some tomorrow.
Tokens:  tf.Tensor([ 421 2724  164 1898   74   99  316  593  123  177   17], shape=(11,), dtype=int32)
Recovered text after detokenizing:  amanda baked cookies and will bring jerry some tomorrow .


In [14]:
def preprocess_batch(text, summ):
    batch_size = tf.shape(summ)[0]    
    
    text = text_tokenizer(text)
    print('Text1',text)
    summ = summ_tokenizer(summ)
    print('Summ1',summ)

    text_start_end_packer = keras_nlp.layers.StartEndPacker(
    sequence_length=MAX_SEQUENCE_LENGTH,
    pad_value=text_tokenizer.token_to_id("[PAD]"),
    )
    text = text_start_end_packer(text)
    print('Text2',text)

    summ_start_end_packer = keras_nlp.layers.StartEndPacker(
    sequence_length=MAX_SEQUENCE_LENGTH + 1,
    start_value=summ_tokenizer.token_to_id("[START]"),
    end_value=summ_tokenizer.token_to_id("[END]"),
    pad_value=summ_tokenizer.token_to_id("[PAD]"),
    )
    summ = summ_start_end_packer(summ)
    # en_de_dict = {
    #     "encoder_inputs": text,
    #     "decoder_inputs": summ[:, :-1],
    # }
    print('Summ2', summ)
    
    return (
    {
        "encoder_inputs": text,
        "decoder_inputs": summ[:, :-1],
    },
        summ[:, 1:],
    )


    

In [15]:
def make_dataset(pairs):
    source_texts, target_texts = zip(*pairs)
    source_texts = list(source_texts)
    target_texts = list(target_texts)
    dataset = tf.data.Dataset.from_tensor_slices((source_texts, target_texts))
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.map(preprocess_batch, num_parallel_calls=tf.data.AUTOTUNE)
    return dataset.shuffle(2048).prefetch(16).cache()
train_ds = make_dataset(text_pairs)


Text1 tf.RaggedTensor(values=Tensor("word_piece_tokenizer_1/FastWordpieceTokenizeWithOffsets/FastWordpieceTokenizeWithOffsets/Cast:0", shape=(None,), dtype=int32, device=/device:CPU:*), row_splits=Tensor("word_piece_tokenizer_1/RaggedFromRowSplits_3/control_dependency:0", shape=(None,), dtype=int64, device=/device:CPU:*))
Summ1 tf.RaggedTensor(values=Tensor("word_piece_tokenizer_1_2/FastWordpieceTokenizeWithOffsets/FastWordpieceTokenizeWithOffsets/Cast:0", shape=(None,), dtype=int32, device=/device:CPU:*), row_splits=Tensor("word_piece_tokenizer_1_2/RaggedFromRowSplits_3/control_dependency:0", shape=(None,), dtype=int64, device=/device:CPU:*))
Text2 Tensor("start_end_packer_1/RaggedToTensor/RaggedTensorToTensor:0", shape=(None, 150), dtype=int32, device=/device:CPU:*)
Summ2 Tensor("start_end_packer_1_2/RaggedToTensor/RaggedTensorToTensor:0", shape=(None, 151), dtype=int32, device=/device:CPU:*)


In [16]:
for inputs, targets in train_ds.take(1):
    print(f'inputs["encoder_inputs"].shape: {inputs["encoder_inputs"].shape}')
    print(f'inputs["decoder_inputs"].shape: {inputs["decoder_inputs"].shape}')
    print(f"targets.shape: {targets.shape}")

inputs["encoder_inputs"].shape: (64, 150)
inputs["decoder_inputs"].shape: (64, 150)
targets.shape: (64, 150)


In [17]:

encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="encoder_inputs")

embedding_layer = keras_nlp.layers.TokenAndPositionEmbedding(
    vocabulary_size=TEXT_VOCAB_SIZE,
    sequence_length=MAX_SEQUENCE_LENGTH,
    embedding_dim=EMBED_DIM,
    mask_zero=True
)
x = embedding_layer(encoder_inputs)

transformer_encoder = keras_nlp.layers.TransformerEncoder(
intermediate_dim=INTERMEDIATE_DIM, num_heads=NUM_HEADS)

encoder_outputs = transformer_encoder(inputs=x)
# encoder = keras.Model(encoder_inputs, encoder_outputs)



In [18]:
decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="decoder_inputs")
encoded_seq_inputs = keras.Input(shape=(None, EMBED_DIM), name="decoder_state_inputs")

x = keras_nlp.layers.TokenAndPositionEmbedding(
vocabulary_size=SUMM_VOCAB_SIZE,
sequence_length=MAX_SEQUENCE_LENGTH,
embedding_dim=EMBED_DIM,
mask_zero=True,)(decoder_inputs)


In [19]:
decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="decoder_inputs")
encoded_seq_inputs = keras.Input(shape=(None, EMBED_DIM), name="decoder_state_inputs")

x = keras_nlp.layers.TokenAndPositionEmbedding(
vocabulary_size=SUMM_VOCAB_SIZE,
sequence_length=MAX_SEQUENCE_LENGTH,
embedding_dim=EMBED_DIM,
mask_zero=True,)(decoder_inputs)

transformer_decoder = keras_nlp.layers.TransformerDecoder(
    intermediate_dim=INTERMEDIATE_DIM, 
    num_heads=NUM_HEADS
)

x = transformer_decoder(decoder_sequence=x, encoder_sequence=encoded_seq_inputs)

x = keras.layers.Dropout(0.5)(x)
decoder_outputs = keras.layers.Dense(SUMM_VOCAB_SIZE, activation="softmax")(x)
decoder = keras.Model(
[
    decoder_inputs,
    encoded_seq_inputs,
],
    decoder_outputs,
)

decoder_outputs = decoder([decoder_inputs, encoder_outputs])

transformer = keras.Model(
[encoder_inputs, decoder_inputs],
decoder_outputs,
name="transformer",)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/layer.py:938: UserWarning: Layer 'functional' (of type Functional) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(


In [20]:
transformer.summary()
# encoder.summary()

Model: "transformer"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ encoder_inputs            │ (None, None)           │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ token_and_position_embed… │ (None, None, 256)      │      2,009,600 │ encoder_inputs[0][0]   │
│ (TokenAndPositionEmbeddi… │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ decoder_inputs            │ (None, None)           │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ transformer_encoder       │ (None, None, 256)      │      1,315,072 │ token_and_position_em… │
│ (TransformerEncoder)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ not_equal (NotEqual)      │ (None, None)           │              0 │ encoder_inputs[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ functional (Functional)   │ (None, None, 3800)     │      3,566,552 │ decoder_inputs[0][0],  │
│                           │                        │                │ transformer_encoder[0… │
│                           │                        │                │ not_equal[0][0]        │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 6,891,224 (26.29 MB)

 Trainable params: 6,891,224 (26.29 MB)

 Non-trainable params: 0 (0.00 B)

In [25]:
transformer.compile(
"adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)
transformer.fit(train_ds, epochs=100)

Epoch 1/100


W0000 00:00:1749087797.809768     121 assert_op.cc:38] Ignoring Assert operator compile_loss/sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/assert_equal_1/Assert/Assert


 79/197 ━━━━━━━━━━━━━━━━━━━━ 6s 55ms/step - accuracy: 0.0641 - loss: 3.5139

W0000 00:00:1749087807.168208     121 assert_op.cc:38] Ignoring Assert operator compile_loss/sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/assert_equal_1/Assert/Assert


197/197 ━━━━━━━━━━━━━━━━━━━━ 32s 80ms/step - accuracy: 0.0653 - loss: 3.4750
Epoch 2/100
197/197 ━━━━━━━━━━━━━━━━━━━━ 11s 55ms/step - accuracy: 0.0709 - loss: 3.2089
Epoch 3/100
197/197 ━━━━━━━━━━━━━━━━━━━━ 11s 55ms/step - accuracy: 0.0757 - loss: 2.9937
Epoch 4/100
197/197 ━━━━━━━━━━━━━━━━━━━━ 11s 55ms/step - accuracy: 0.0808 - loss: 2.7959
Epoch 5/100
197/197 ━━━━━━━━━━━━━━━━━━━━ 11s 55ms/step - accuracy: 0.0844 - loss: 2.6389
Epoch 6/100
197/197 ━━━━━━━━━━━━━━━━━━━━ 11s 55ms/step - accuracy: 0.0889 - loss: 2.4778
Epoch 7/100
197/197 ━━━━━━━━━━━━━━━━━━━━ 11s 55ms/step - accuracy: 0.0934 - loss: 2.3204
Epoch 8/100
197/197 ━━━━━━━━━━━━━━━━━━━━ 11s 55ms/step - accuracy: 0.0978 - loss: 2.1674
Epoch 9/100
197/197 ━━━━━━━━━━━━━━━━━━━━ 11s 55ms/step - accuracy: 0.1018 - loss: 2.0308
Epoch 10/100
197/197 ━━━━━━━━━━━━━━━━━━━━ 11s 55ms/step - accuracy: 0.1055 - loss: 1.9106
Epoch 11/100
197/197 ━━━━━━━━━━━━━━━━━━━━ 11s 55ms/step - accuracy: 0.1093 - loss: 1.7890
Epoch 12/100
197/197 ━━━━━━━━━━

In [26]:
@tf.function

def decode_sequences(input_sentences):
    batch_size = tf.shape(input_sentences)[0]
    
    encoder_input_tokens = text_tokenizer.tokenize(input_sentences).to_tensor(
    shape=(None, MAX_SEQUENCE_LENGTH)
    )
    
    def next(prompt, cache, index):
        logits = transformer([encoder_input_tokens, prompt])[:, index - 1, :]
        hidden_states = None
        return logits, hidden_states, cache
    
    length = MAX_SEQUENCE_LENGTH
    start = tf.fill((batch_size, 1), summ_tokenizer.token_to_id("[START]"))
    pad = tf.fill((batch_size, length - 1), summ_tokenizer.token_to_id("[PAD]"))
    prompt = tf.concat((start, pad), axis=-1)
    
    generated_tokens = keras_nlp.samplers.GreedySampler()(
    next,
    prompt,
    index=1, 
    )
    generated_sentences = summ_tokenizer.detokenize(generated_tokens)
    return generated_sentences

In [31]:
def truncate_at_end(text):
    if "[END]" in text:
        end_position = text.find("[END]")
        return text[:end_position].strip()
    return text.strip()

for i in range(4):
    choice = random.randint(0, len(test_texts))
    input_sentence = test_texts[choice]
    summarized = decode_sequences(tf.constant([input_sentence]))
    summarized = summarized.numpy()[0].decode("utf-8")
    
    summarized = truncate_at_end(summarized)
    
    summarized = (
        summarized.replace("[PAD]", "")
        .replace("[START]", "")
        .replace("[END]", "")  
        .strip()
    )
    print(f"** Example {i} **")
    print("Text:", input_sentence)
    print("Prediction:", summarized)
    print("Ground Truth:", original[choice])
    print()

** Example 0 **
Text: tom: hey, you forgot your scarf yesterday
tom: so you have an excuse to drop by :d
jim: haha. sure. i'll handle myself without a scarf until tomorrow.
jim: i'll visit you "to get my scarf" after work if you know what i mean :d
tom: hahaha. sure. see you then.
Prediction: jim has forgot his scarf yesterday . he ' ll visit tom tomorrow .
Ground Truth: jim has forgot his scarf yesterday. he'll visit tom tomorrow after work to get it.

** Example 1 **
Text: aretha: i've put some plants on your terrace this morning. in the shade. all perennials.
lucia: thanks a lot! you don't want them any more?
aretha: they're from dorothea. i just have no more space. happy planting!
lucia: ta!
Prediction: aretha has put some plants from dorothea on lucia ' s terrace this morning .
Ground Truth: aretha has put some plants from dorothea on lucia's terrace this morning.

** Example 2 **
Text: ellen: let's go shopping tomorrow
portia: nooooo, it's too close to christmas
ellen: we could m